In [1]:
import os
import csv
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pyplot import plot, ion, show, savefig, cla, figure

## Helper functions to load and process original csv files

In [2]:
# this function load one .cvs (a sequence)
#for scada files ONLY
def load_data(dataset, csv_folder='./NAB-known-anomaly/csv-files/'):
    filename = dataset + '.csv'
    data_file = os.path.join(csv_folder, filename)
    t_unit = 'time unit'        
    
    t = []
    readings = []
    idx_anomaly = []
    i = 0
    j = 0
    with open(data_file) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        #print("\n--> Anomalies occur at:")
        for row in readCSV:
            if i > 0:
                if row[-1] == '1':
                    if j < 205972:
                        j = j + 1
                        continue
                    idx_anomaly.append(i)
                t.append(i)
                readings.append(np.array(row[:-1]).astype(np.float))
            j = j + 1        
            i = i + 1
    t = np.asarray(t)
    readings = np.asarray(readings)
    print("\nOriginal csv file contains {} timestamps.".format(t.shape))
    print("Processed time series contain {} readings.".format(readings.shape))
    #print("Anomaly indices are {}".format(idx_anomaly))
    
    return t, t_unit, readings, idx_anomaly

In [3]:
t, t_unit, readings, idx_anomaly = load_data('scada2')



Original csv file contains (229527,) timestamps.
Processed time series contain (229527, 16) readings.


In [4]:
np.max(readings)

1.0

In [7]:
a = np.array((1,2,3,0,5,6,7,0.4))
a = a[a<(2+1)]
#np.arange(3,30)
a

array([1. , 2. , 0. , 0.4])

In [8]:
def find_runs(x):
    """Find runs of consecutive items in an array."""

    # ensure array
    x = np.asanyarray(x)
    if x.ndim != 1:
        raise ValueError('only 1D array supported')
    n = x.shape[0]

    # handle empty array
    if n == 0:
        return np.array([]), np.array([]), np.array([])

    else:
        # find run starts
        loc_run_start = np.empty(n, dtype=bool)
        loc_run_start[0] = True
        np.not_equal(x[:-1], x[1:], out=loc_run_start[1:])
        run_starts = np.nonzero(loc_run_start)[0]

        # find run values
        run_values = x[loc_run_start]

        # find run lengths
        run_lengths = np.diff(np.append(run_starts, n))

        return run_values, run_starts, run_lengths

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('./NAB-known-anomaly/csv-files/scada1.csv')
df

,address,function,length,setpoint,gain,reset rate,deadband,cycle time,rate,system mode,control scheme,pump,solenoid,pressure measurement,crc rate,command respond,binary result
0,-0.013693,-0.473801,-0.823855,-0.088897,0.232286,-1.318181,-0.003811,-0.140748,-0.139904,-0.619606,0.719006,-0.685442,-0.913608,-0.053623,-0.826166,0.997809,0
1,-0.013693,-0.473801,0.178277,-0.088897,0.232286,-1.318181,-0.003811,-0.140748,-0.139904,-0.619606,0.719006,-0.685442,-0.913608,-0.053623,-1.015153,-1.002193,0
2,-0.013693,0.267787,1.648072,-0.088897,0.232286,-1.318181,-0.003811,-0.140748,-0.139904,-0.619606,0.719006,-0.685442,-0.913608,-0.053623,0.776356,0.997809,0
3,-0.013693,0.267787,-0.823855,-0.088897,0.232286,-1.318181,-0.003811,-0.140748,-0.139904,-0.619606,0.719006,-0.685442,-0.913608,-0.053623,0.960185,-1.002193,0
4,-0.013693,-0.473801,-0.823855,-0.088897,0.232286,-1.318181,-0.003811,-0.140748,-0.139904,-0.619606,0.719006,-0.685442,-0.913608,-0.053623,-0.826166,0.997809,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274623,-0.013693,-0.473801,0.178277,0.059974,0.579894,0.452878,-0.003673,-0.630065,-0.139904,1.894931,0.719006,1.458907,1.094558,-0.053623,-0.921212,-1.002193,1
274624,-0.013693,0.267787,1.648072,-0.237768,0.579894,0.452878,-0.003673,-0.630065,-0.139904,1.894931,0.719006,1.458907,1.094558,-0.053623,-0.267309,0.997809,1
274625,-0.013693,0.267787,-0.823855,-0.237768,0.579894,0.452878,-0.003673,-0.630065,-0.139904,1.894931,0.719006,1.458907,1.094558,-0.053623,0.960185,-1.002193,1
274626,-0.013693,-0.473801,-0.823855,-0.237768,0.579894,0.452878,-0.003673,-0.630065,-0.139904,1.894931,0.719006,1.458907,1.094558,-0.053623,-0.826166,0.997809,0


In [11]:
run_vals, run_starts, run_lengths = find_runs(df['binary result'])
run_lengths = run_lengths[run_vals==0]
run_starts = run_starts[run_vals==0]
arg = np.argmax(run_lengths)
print(arg)
train_start = run_starts[arg]
train_end = train_start + run_lengths[arg]
print(train_start,train_end)
#print(df['binary result'][train_start:train_end])
#readings[train_start:train_end]
np.std(np.array(readings), axis=0)

18970
162195 163611


array([0.00950141, 0.06644603, 0.37317608, 0.13186913, 0.05908807,
       0.07057748, 0.00417454, 0.06935928, 0.06757354, 0.39569464,
       0.47477088, 0.46336916, 0.49821839, 0.03527813, 0.03669692,
       0.49885847])

In [13]:
x = np.min(readings, axis=0)
print(x)
x[x==0] = 1
x

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.00342136 0.        ]


array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.00342136,
       1.        ])

In [13]:
# This function plots a dataset with the train/test split and known anomalies
# Relies on helper function load_data()

def process_and_save_specified_dataset(dataset, idx_split, pi_num, y_scale=5, save_file=False):
    t, t_unit, readings, idx_anomaly = load_data(dataset)
    
    # split into training and test sets
    #[0,40217]
#[40217, 80434]
#[80434, 120651]
#[120651,160869]
    split_fl = [[0,40217],[40217, 80434],[80434, 120651],[120651,160869]]
    training = readings[split_fl[pi_num-1][0]:split_fl[pi_num-1][1]]
    t_train = t[split_fl[pi_num-1][0]:split_fl[pi_num-1][1]]
    
    # normalise by training mean and std 
    train_m = np.mean(training, axis=0)
    train_std = np.std(training, axis=0)
    #train_std[train_std==0] = 0.2 #avoid divide by zero
    #print("\nTraining set mean is {}".format(train_m))
    #print("Training set std is {}".format(train_std))
    #readings_normalised = (readings - train_m) / train_std
    #training = readings_normalised[split_fl[pi_num-1][0]:split_fl[pi_num-1][1]]
    
    #training = readings_normalised[idx_split[0]:idx_split[1]] 
    #160869
    
    if idx_split[0] == 0:
        test = readings[idx_split[1]:]
        t_test = t[idx_split[1]:] - idx_split[1]
        idx_anomaly_test = np.asarray(idx_anomaly) - idx_split[1]
    else:
        test = [training[:idx_split[0]], training[idx_split[1]:]]
        t_test = [t[:idx_split[0]], t[idx_split[1]:] - idx_split[1]]
        idx_anomaly_split = np.squeeze(np.argwhere(np.asarray(idx_anomaly)>idx_split[0]))
        idx_anomaly_test = [np.asarray(idx_anomaly[:idx_anomaly_split[0]]), 
                            np.asarray(idx_anomaly[idx_anomaly_split[0]:]) - idx_split[1]]
    print("Anomaly indices in the test set are {}".format(idx_anomaly_test))
    
    #if dataset = scada
    #idx_test = [0,3000]
    #t_test = np.arange(idx_test[1] - idx_test[0]) + 1
    #test = readings_normalised[idx_test[0]:idx_test[1]]
    #idx_anomaly_test = np.asarray(idx_anomaly) - idx_test[0] #+ idx_test[0]    
    #idx_anomaly_test = idx_anomaly_test[idx_anomaly_test < (idx_test[1] + 1)]
    #print("Anomaly indices in the test set are {}".format(idx_anomaly_test))
    
    if save_file:
        save_dir = './NAB-known-anomaly/'
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        np.savez(save_dir+dataset+'_'+ str(pi_num)+'.npz', t=t, t_unit=t_unit, idx_anomaly=idx_anomaly,
                    idx_split=idx_split, training=training, test=test, train_m=train_m, train_std=train_std,
                    t_train=t_train, t_test=t_test, idx_anomaly_test=idx_anomaly_test) #deleted readings = readings
        print("\nProcessed time series are saved at {}".format(save_dir+dataset+'.npz'))
    else:
        print("\nProcessed time series are not saved.")
        
    print("t_train:",t_train,"t_test:",t_test)

    # plot the whole normalised sequence
    #fig, axs = plt.subplots(1, 1, figsize=(18, 4), edgecolor='k')
    #fig.subplots_adjust(hspace=.4, wspace=.4)
    # axs = axs.ravel()
    # for i in range(4):
    #axs.plot(t, training)
    #axs.plot(t_test,test)
    #if idx_split[0] == 0:
    #    axs.plot(idx_split[1]*np.ones(20), np.linspace(-y_scale,y_scale,20), 'b--')
    #else:
    #    for i in range(2):
    #        axs.plot(idx_split[i]*np.ones(20), np.linspace(-y_scale,y_scale,20), 'b--')
    #for j in range(len(idx_anomaly)):
    #    axs.plot(idx_anomaly[j]*np.ones(20), np.linspace(-y_scale,y_scale,20), 'r--')
    #     axs.plot(data[:,1])
    #axs.grid(True)
    #axs.set_xlim(0, len(t))
    #axs.set_ylim(-y_scale, y_scale)
    #axs.set_xlabel("timestamp (every {})".format(t_unit))
    #axs.set_ylabel("normalised readings")
    #axs.set_title("{} dataset\n(normalised by train mean {:.2f} and std {:.2f})".format(dataset, np.mean(train_m), np.mean(train_std)))
    #axs.legend(('data', 'train test set split', 'anomalies'))
    
    return t, training

## Example on ambient temperature series

In [14]:
dataset = ['scada1','scada2','scada3','scada4']
idx_split = [0,160869]
#40217.25
#[0,40217]
#[40217, 80434]
#[80434, 120651]
#[120651,160869]
for name in dataset:
    t, training = process_and_save_specified_dataset(name, idx_split, 1, save_file=True)
    t, training = process_and_save_specified_dataset(name, idx_split, 2, save_file=True)
    t, training = process_and_save_specified_dataset(name, idx_split, 3, save_file=True)
    t, training = process_and_save_specified_dataset(name, idx_split, 4, save_file=True)


Original csv file contains (229527,) timestamps.
Processed time series contain (229527, 16) readings.
Anomaly indices in the test set are [    3     7    11 ... 68654 68655 68656]

Processed time series are saved at ./NAB-known-anomaly/scada1.npz
t_train: [    1     2     3 ... 40215 40216 40217] t_test: [    1     2     3 ... 68656 68657 68658]

Original csv file contains (229527,) timestamps.
Processed time series contain (229527, 16) readings.
Anomaly indices in the test set are [    3     7    11 ... 68654 68655 68656]

Processed time series are saved at ./NAB-known-anomaly/scada1.npz
t_train: [40218 40219 40220 ... 80432 80433 80434] t_test: [    1     2     3 ... 68656 68657 68658]

Original csv file contains (229527,) timestamps.
Processed time series contain (229527, 16) readings.
Anomaly indices in the test set are [    3     7    11 ... 68654 68655 68656]

Processed time series are saved at ./NAB-known-anomaly/scada1.npz
t_train: [ 80435  80436  80437 ... 120649 120650 12065